In [47]:
#Car Price Prediction

In [48]:
import findspark
findspark.init()

In [15]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TP1 BDM").getOrCreate()
df = spark.read.csv("./CarPrice_Assignment.csv",header=True)
df.take(1)

[Row(car_ID='1', symboling='3', CarName='alfa-romero giulia', fueltype='gas', aspiration='std', doornumber='two', carbody='convertible', drivewheel='rwd', enginelocation='front', wheelbase='88.6', carlength='168.8', carwidth='64.1', carheight='48.8', curbweight='2548', enginetype='dohc', cylindernumber='four', enginesize='130', fuelsystem='mpfi', boreratio='3.47', stroke='2.68', compressionratio='9', horsepower='111', peakrpm='5000', citympg='21', highwaympg='27', price='13495')]

In [16]:
df.cache()
df.printSchema()

root
 |-- car_ID: string (nullable = true)
 |-- symboling: string (nullable = true)
 |-- CarName: string (nullable = true)
 |-- fueltype: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- doornumber: string (nullable = true)
 |-- carbody: string (nullable = true)
 |-- drivewheel: string (nullable = true)
 |-- enginelocation: string (nullable = true)
 |-- wheelbase: string (nullable = true)
 |-- carlength: string (nullable = true)
 |-- carwidth: string (nullable = true)
 |-- carheight: string (nullable = true)
 |-- curbweight: string (nullable = true)
 |-- enginetype: string (nullable = true)
 |-- cylindernumber: string (nullable = true)
 |-- enginesize: string (nullable = true)
 |-- fuelsystem: string (nullable = true)
 |-- boreratio: string (nullable = true)
 |-- stroke: string (nullable = true)
 |-- compressionratio: string (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- peakrpm: string (nullable = true)
 |-- citympg: string (nullable = true)
 |

In [10]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
car_ID,205,103.0,59.32256456582661,1,99
symboling,205,0.8341463414634146,1.2453068281055295,-1,3
CarName,205,None,None,Nissan versa,vw rabbit
fueltype,205,None,None,diesel,gas
aspiration,205,None,None,std,turbo
doornumber,205,None,None,four,two
carbody,205,None,None,convertible,wagon
drivewheel,205,None,None,4wd,rwd
enginelocation,205,None,None,front,rear


In [16]:
columns_to_drop=['CarName','fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','enginetype','cylindernumber','fuelsystem']
df=df.drop(*columns_to_drop)
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
car_ID,205,103.0,59.32256456582661,1,99
symboling,205,0.8341463414634146,1.2453068281055295,-1,3
wheelbase,205,98.75658536585378,6.0217756850255695,100.4,99.8
carlength,205,174.04926829268305,12.337288526555188,141.1,208.1
carwidth,205,65.90780487804875,2.145203852687182,60.3,72.3
carheight,205,53.724878048780525,2.443521969904905,47.8,59.8
curbweight,205,2555.5658536585365,520.6802035016384,1488,4066
enginesize,205,126.90731707317073,41.642693438179855,103,98
boreratio,205,3.3297560975609772,0.27084370542622915,2.54,3.94


In [26]:
df=df.withColumn('car_ID',df['car_ID'].cast("float").alias('car_ID'))
df=df.withColumn('symboling',df['symboling'].cast("float").alias('symboling'))
df=df.withColumn('wheelbase',df['wheelbase'].cast("float").alias('wheelbase'))
df=df.withColumn('carlength',df['carlength'].cast("float").alias('carlength'))
df=df.withColumn('carwidth',df['carwidth'].cast("float").alias('carwidth'))
df=df.withColumn('carheight',df['carheight'].cast("float").alias('carheight'))
df=df.withColumn('curbweight',df['curbweight'].cast("float").alias('curbweight'))
df=df.withColumn('enginesize',df['enginesize'].cast("float").alias('enginesize'))
df=df.withColumn('boreratio',df['boreratio'].cast("float").alias('boreratio'))
df=df.withColumn('stroke',df['stroke'].cast("float").alias('stroke'))
df=df.withColumn('compressionratio',df['compressionratio'].cast("float").alias('compressionratio'))
df=df.withColumn('horsepower',df['horsepower'].cast("float").alias('horsepower'))
df=df.withColumn('peakrpm',df['peakrpm'].cast("float").alias('peakrpm'))
df=df.withColumn('citympg',df['citympg'].cast("float").alias('citympg'))
df=df.withColumn('highwaympg',df['highwaympg'].cast("float").alias('highwaympg'))
df=df.withColumn('price',df['price'].cast("float").alias('price'))
df.printSchema()

root
 |-- car_ID: float (nullable = true)
 |-- symboling: float (nullable = true)
 |-- wheelbase: float (nullable = true)
 |-- carlength: float (nullable = true)
 |-- carwidth: float (nullable = true)
 |-- carheight: float (nullable = true)
 |-- curbweight: float (nullable = true)
 |-- enginesize: float (nullable = true)
 |-- boreratio: float (nullable = true)
 |-- stroke: float (nullable = true)
 |-- compressionratio: float (nullable = true)
 |-- horsepower: float (nullable = true)
 |-- peakrpm: float (nullable = true)
 |-- citympg: float (nullable = true)
 |-- highwaympg: float (nullable = true)
 |-- price: float (nullable = true)



In [27]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation entre price et ", i, df.stat.corr('price',i))

Correlation entre price et  car_ID -0.10909334037698282
Correlation entre price et  symboling -0.07997822501359858
Correlation entre price et  wheelbase 0.577815609013954
Correlation entre price et  carlength 0.6829200061793368
Correlation entre price et  carwidth 0.7593252732789826
Correlation entre price et  carheight 0.11933627096290873
Correlation entre price et  curbweight 0.8353048796203731
Correlation entre price et  enginesize 0.8741448022848783
Correlation entre price et  boreratio 0.5531732639743967
Correlation entre price et  stroke 0.07944309329818429
Correlation entre price et  compressionratio 0.06798351616221464
Correlation entre price et  horsepower 0.8081388231007026
Correlation entre price et  peakrpm -0.0852671497816066
Correlation entre price et  citympg -0.6857513366309157
Correlation entre price et  highwaympg -0.6975990921640883
Correlation entre price et  price 1.0


In [28]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler=VectorAssembler(inputCols=['car_ID','symboling','wheelbase','carlength','carwidth','carheight','curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg','price'],outputCol='features')
vect_df=vectorAssembler.transform(df)
vect_df=vect_df.select(['features','price'])
vect_df.show(5)

+--------------------+-------+
|            features|  price|
+--------------------+-------+
|[1.0,3.0,88.59999...|13495.0|
|[2.0,3.0,88.59999...|16500.0|
|[3.0,1.0,94.5,171...|16500.0|
|[4.0,2.0,99.80000...|13950.0|
|[5.0,2.0,99.40000...|17450.0|
+--------------------+-------+
only showing top 5 rows



In [32]:
splits=vect_df.randomSplit([0.7,0.3])
train_df=splits[0]
test_df=splits[1]
from pyspark.ml.regression import LinearRegression
linear=LinearRegression(featuresCol='features',labelCol='price',maxIter=10, regParam=0.3,elasticNetParam=0.8)
linear_model=linear.fit(train_df)
print('Les coefficients sont : '+str(linear_model.coefficients))
print('Le B est égale à : '+str(linear_model.intercept))

Les coefficients sont : [-3.440437041607438,46.438956872028825,3.076044650919893,-5.8316961621322285,-71.46837930733722,123.19543961390345,-0.0,33.71660558845025,-28.34755631544181,-647.3549230139257,76.95541633071758,6.024055078661431,1.0236883483044283,-5.263277452743497,18.589976502584403,0.8089529018400144]
Le B est égale à : -7531.121902276664


In [33]:
synthese=linear_model.summary
print('RMSE : '+str(synthese.rootMeanSquaredError))
print('R² : '+str(synthese.r2))

RMSE : 712.3417714301808
R² : 0.9928507572232271


In [34]:
train_df.describe().toPandas()

,summary,price
0,count,150
1,mean,13645.607786458333
2,stddev,8453.000304787609
3,min,5118.0
4,max,45400.0


In [38]:
linear_predictions=linear_model.transform(test_df)
linear_predictions.select("prediction",'price','features').show(5)
from pyspark.ml.evaluation import RegressionEvaluator
linear_evaluator=RegressionEvaluator(predictionCol="prediction",labelCol='price',metricName='r2')
print("R carré sur les données de test = %g"%linear_evaluator.evaluate(linear_predictions))

+------------------+-------+--------------------+
|        prediction|  price|            features|
+------------------+-------+--------------------+
|13661.528127333575|13495.0|[1.0,3.0,88.59999...|
|16088.991160321213|16500.0|[2.0,3.0,88.59999...|
| 16843.99394062676|16500.0|[3.0,1.0,94.5,171...|
|13897.653552800188|13950.0|[4.0,2.0,99.80000...|
| 35581.85693733253|36880.0|[18.0,0.0,110.0,1...|
+------------------+-------+--------------------+
only showing top 5 rows

R carré sur les données de test = 0.988887


In [39]:
resultat_test=linear_model.evaluate(test_df)
print("RMSE sur les données de test = %g"%resultat_test.rootMeanSquaredError)

RMSE sur les données de test = 681.277


In [40]:
print("Le nombre d'itérations effectués = %d"%synthese.totalIterations)
print("Les residus : ")
synthese.residuals.show(10)

Le nombre d'itérations effectués = 10
Les residus : 
+------------------+
|         residuals|
+------------------+
| 22.36606642804327|
| -309.821557698142|
|325.56571553372123|
| 560.1731413489069|
|1569.5876878808376|
| 572.4201243202369|
| -94.2870726282672|
| 96.59959174659161|
| 690.6862697634679|
| 719.9180650566741|
+------------------+
only showing top 10 rows



In [41]:
predictions=linear_model.transform(test_df)
predictions.select("price",'features','prediction').show()

+-------+--------------------+------------------+
|  price|            features|        prediction|
+-------+--------------------+------------------+
|13495.0|[1.0,3.0,88.59999...|13661.528127333575|
|16500.0|[2.0,3.0,88.59999...|16088.991160321213|
|16500.0|[3.0,1.0,94.5,171...| 16843.99394062676|
|13950.0|[4.0,2.0,99.80000...|13897.653552800188|
|36880.0|[18.0,0.0,110.0,1...| 35581.85693733253|
| 6295.0|[20.0,1.0,94.5,15...|7004.2307736647235|
| 6479.0|[31.0,2.0,86.5999...| 6370.714575537322|
| 5399.0|[33.0,1.0,93.6999...| 6032.388263715127|
| 7295.0|[37.0,0.0,96.5,15...|  8832.81811433279|
| 8845.0|[40.0,0.0,96.5,17...| 9686.964724700341|
| 6785.0|[44.0,0.0,94.3000...|7283.3035058142705|
| 8916.5|[45.0,1.0,94.5,15...| 9038.889879798135|
| 6795.0|[53.0,1.0,93.0999...|  6972.05276281945|
|10945.0|[56.0,3.0,95.3000...| 9962.790694464828|
|10795.0|[64.0,0.0,98.8000...|11639.747787391007|
|18280.0|[66.0,0.0,104.900...| 17736.36603282481|
|28248.0|[69.0,-1.0,110.0,...|27378.574256991156|
